In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast 

# import os
# if os.name == 'posix' :
#     plt.rc('font', family='AppleGothic')
# else :
#     plt.rc('font', family='Malgun Gothic')
    
plt.rc('axes', unicode_minus=False)

plt.rcParams['font.family'] = 'NanumGothic' 
plt.rcParams['axes.unicode_minus'] = False
import koreanize_matplotlib

## 데이터 불러오기

In [2]:
user_df = pd.read_parquet('./votes/accounts_user.parquet')
user_df.head()

,id,is_superuser,is_staff,gender,point,friend_id_list,is_push_on,created_at,block_user_id_list,hide_user_id_list,ban_status,report_count,alarm_count,pending_chat,pending_votes,group_id
0,831956,1,1,None,600,"[1292473, 913158, 1488461, 1064695, 1043565, 1...",0,2023-03-29 03:44:14.047130,[],[],N,0,0,0,0,NaN
1,831962,0,0,F,2248,"[833025, 832642, 982531, 879496, 838541, 83752...",1,2023-03-29 05:18:56.162368,[],[],N,253,40878,5499,110,12.0
2,832151,0,0,M,1519,"[838785, 982531, 882567, 879496, 838541, 83649...",0,2023-03-29 12:56:34.989468,[],[],N,0,37,0,47,1.0
3,832340,0,0,F,57,"[841345, 982531, 838785, 963714, 882567, 83252...",1,2023-03-29 12:56:35.020790,[],[],N,0,19,0,21,1.0
4,832520,0,0,M,1039,"[874050, 849763, 874212, 844297, 838541, 84004...",0,2023-03-29 12:56:35.049311,[],[],N,0,29,0,15,12.0


In [3]:
userquestion_df = pd.read_parquet('./votes/accounts_userquestionrecord.parquet')
userquestion_df.head()

,id,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,report_count,opened_times
0,771777,C,2023-04-28 12:27:49,849469,252,849436,998458,0,N,2023-04-28 12:27:49,0,0
1,771800,C,2023-04-28 12:28:02,849446,244,849436,998459,0,N,2023-04-28 12:28:02,0,0
2,771812,C,2023-04-28 12:28:09,849454,183,849436,998460,1,N,2023-04-28 12:28:09,0,0
3,771828,C,2023-04-28 12:28:16,847375,101,849436,998461,0,N,2023-04-28 12:28:16,0,0
4,771851,C,2023-04-28 12:28:26,849477,209,849436,998462,1,N,2023-04-28 12:28:26,0,0


In [4]:
userquestion_567_df = userquestion_df[(userquestion_df['created_at']>='2023-05-01') & (userquestion_df['created_at']<'2023-08-01')]

In [5]:
school_df = pd.read_parquet('./votes/accounts_school.parquet')
school_df.head()

,id,address,student_count,school_type
0,4,충청북도 충주시,239,H
1,5,충청북도 충주시,160,M
2,6,충청북도 충주시,200,H
3,7,충청북도 충주시,114,H
4,8,충청북도 충주시,139,M


In [6]:
group_df = pd.read_parquet('./votes/accounts_group.parquet')
group_df.head()

,id,grade,class_num,school_id
0,1,1,1,1
1,8,3,2,1
2,9,3,3,1
3,10,2,2,1
4,11,2,3,1


In [8]:
group_df.describe()

,id,grade,class_num,school_id
count,84515.000000,84515.000000,84515.000000,84515.000000
mean,42279.376087,2.113992,5.318606,3079.496468
std,24400.962630,0.754896,4.089901,1675.036093
min,1.000000,1.000000,0.000000,1.000000
25%,21147.500000,2.000000,2.000000,1752.000000
50%,42279.000000,2.000000,4.000000,3023.000000
75%,63411.500000,3.000000,7.000000,4620.000000
max,84546.000000,20.000000,41.000000,5965.000000


In [ ]:
# 3학년 이상인 데이터 수
group_df[group_df['grade'] > 3]

,id,grade,class_num,school_id
32624,32644,4,10,4658
61622,61647,20,7,3867


In [11]:
# 0반 데이터 수 
group_df[group_df['class_num'] == 0]

,id,grade,class_num,school_id
62234,62259,2,0,4348
62626,62651,2,0,1820


In [15]:
# 20반 보다 큰 수 확인
group_df[group_df['class_num'] > 20]

,id,grade,class_num,school_id
70368,70394,1,41,736


- 이상치 제거
    - grade : 4학년, 20학년 제외
    - class_num : 0반, 41반 제외

In [ ]:
# 이상치 제거 하기
group_df = group_df[(group_df['grade'] <= 3) & (group_df['class_num'] <= 20) & (group_df['class_num'] > 0) ]

---

## 데이터 전처리 (합치기, 추출하기 등)

### 10개 학교 데이터로 뽑기

In [18]:
# school_id, group_id 테이블 합치기
school_group_df = pd.merge(group_df, school_df, left_on='school_id', right_on='id', how='left')
school_group_df = school_group_df.drop(columns=['class_num', 'id_y'])
school_group_df = school_group_df.rename(columns={'id_x':'group_id'})
school_group_df.head()

,group_id,grade,school_id,address,student_count,school_type
0,1,1,1,NaN,NaN,NaN
1,8,3,1,NaN,NaN,NaN
2,9,3,1,NaN,NaN,NaN
3,10,2,1,NaN,NaN,NaN
4,11,2,1,NaN,NaN,NaN


In [22]:
user_group_df = user_df[['id', 'group_id']]
user_school_df = pd.merge(user_group_df, school_group_df, on='group_id', how='left')
user_school_df.head()

,id,group_id,grade,school_id,address,student_count,school_type
0,831956,NaN,NaN,NaN,NaN,NaN,NaN
1,831962,12.0,2.0,1.0,NaN,NaN,NaN
2,832151,1.0,1.0,1.0,NaN,NaN,NaN
3,832340,1.0,1.0,1.0,NaN,NaN,NaN
4,832520,12.0,2.0,1.0,NaN,NaN,NaN


In [23]:
# 결측치를 0으로 채워서 사라지지 않게 하기
user_school_fillna_df = user_school_df.copy()
user_school_fillna_df = user_school_fillna_df.fillna(0)

In [25]:
user_school_fillna_df.head()

,id,group_id,grade,school_id,address,student_count,school_type
0,831956,0.0,0.0,0.0,0,0.0,0
1,831962,12.0,2.0,1.0,0,0.0,0
2,832151,1.0,1.0,1.0,0,0.0,0
3,832340,1.0,1.0,1.0,0,0.0,0
4,832520,12.0,2.0,1.0,0,0.0,0


In [48]:
# 투표 기록 테이블에 학교 정보 합치기
userquestion_school_df = pd.merge(userquestion_df, user_school_fillna_df, left_on='user_id', right_on='id', how='left')
userquestion_school_df.head()

,id_x,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,report_count,opened_times,id_y,group_id,grade,school_id,address,student_count,school_type
0,771777,C,2023-04-28 12:27:49,849469,252,849436,998458,0,N,2023-04-28 12:27:49,0,0,849436,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
1,771800,C,2023-04-28 12:28:02,849446,244,849436,998459,0,N,2023-04-28 12:28:02,0,0,849436,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
2,771812,C,2023-04-28 12:28:09,849454,183,849436,998460,1,N,2023-04-28 12:28:09,0,0,849436,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
3,771828,C,2023-04-28 12:28:16,847375,101,849436,998461,0,N,2023-04-28 12:28:16,0,0,849436,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
4,771851,C,2023-04-28 12:28:26,849477,209,849436,998462,1,N,2023-04-28 12:28:26,0,0,849436,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H


In [29]:
# 필요없는 컬럼 삭제하기
userquestion_school_df = userquestion_school_df.drop(columns=['id_x','report_count', 'opened_times', 'id_y'])
userquestion_school_df.head()

,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,group_id,grade,school_id,address,student_count,school_type
0,C,2023-04-28 12:27:49,849469,252,849436,998458,0,N,2023-04-28 12:27:49,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
1,C,2023-04-28 12:28:02,849446,244,849436,998459,0,N,2023-04-28 12:28:02,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
2,C,2023-04-28 12:28:09,849454,183,849436,998460,1,N,2023-04-28 12:28:09,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
3,C,2023-04-28 12:28:16,847375,101,849436,998461,0,N,2023-04-28 12:28:16,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H
4,C,2023-04-28 12:28:26,849477,209,849436,998462,1,N,2023-04-28 12:28:26,356.0,2.0,271.0,충청남도 천안시 서북구,491.0,H


In [46]:
# 학교 수 확인해보기
userquestion_school_df['school_id'].nunique()

5552

In [30]:
# 5~7월 데이터로만 추출
userquestion_school_567_df = userquestion_school_df[(userquestion_school_df['created_at'].dt.year == 2023)&(userquestion_school_df['created_at'].dt.month.isin([5,6,7]))]
userquestion_school_567_df.head()

,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,group_id,grade,school_id,address,student_count,school_type
30798,C,2023-05-01 03:37:42,855817,202,857913,1821408,0,N,2023-05-01 03:37:42,5691.0,1.0,271.0,충청남도 천안시 서북구,491.0,H
30799,C,2023-05-01 03:38:24,855549,116,856430,1861799,0,N,2023-05-01 03:38:24,4296.0,1.0,271.0,충청남도 천안시 서북구,491.0,H
30800,C,2023-05-01 03:38:38,855509,199,856430,1861800,0,N,2023-05-01 03:38:38,4296.0,1.0,271.0,충청남도 천안시 서북구,491.0,H
30801,C,2023-05-01 03:38:48,855509,238,856430,1861801,0,N,2023-05-01 03:38:48,4296.0,1.0,271.0,충청남도 천안시 서북구,491.0,H
30802,C,2023-05-01 03:38:53,860439,190,856430,1861802,0,N,2023-05-01 03:38:53,4296.0,1.0,271.0,충청남도 천안시 서북구,491.0,H


In [63]:
# 학교 수 확인해보기
userquestion_school_567_df['school_id'].nunique()

10

In [64]:
# 학교 리스트 담기
school_list = list(userquestion_school_567_df['school_id'].unique())
school_list

[271.0, 4516.0, 369.0, 352.0, 1478.0, 5520.0, 5491.0, 5372.0, 1719.0, 4426.0]

In [65]:
user_school_fillna_df.head()

,id,group_id,grade,school_id,address,student_count,school_type
0,831956,0.0,0.0,0.0,0,0.0,0
1,831962,12.0,2.0,1.0,0,0.0,0
2,832151,1.0,1.0,1.0,0,0.0,0
3,832340,1.0,1.0,1.0,0,0.0,0
4,832520,12.0,2.0,1.0,0,0.0,0


In [56]:
# 10개 학교의 학생 수 
user_school_10_df = user_school_fillna_df[user_school_fillna_df['school_id'].isin(school_list)]
user_school_10_df['id'].nunique()

5090

In [57]:
# 학생 user_id 리스트에 담기
user_id_list = list(user_school_10_df['id'].unique())

In [66]:
# 유저 수 확인해보기
userquestion_school_567_df['user_id'].nunique()

4835

### 10개 학교 학생들의 누적 회원가입 수 구하기

In [58]:
# 전체 user 데이터에서 학생들 user_id가 일치하는 것들만 남기기
user_target_df = user_df[user_df['id'].isin(user_id_list)]
user_target_df.head()

,id,is_superuser,is_staff,gender,point,friend_id_list,is_push_on,created_at,block_user_id_list,hide_user_id_list,ban_status,report_count,alarm_count,pending_chat,pending_votes,group_id
396,838023,0,0,F,2456,"[855552, 861830, 859783, 850186, 868492, 85595...",1,2023-04-19 09:06:00.719792,[],[],N,0,1,0,115,233.0
633,838466,0,0,F,1680,"[876288, 869250, 873349, 867975, 878602, 11573...",1,2023-04-19 15:10:56.560337,[],[],N,0,1,0,244,321.0
756,838642,0,0,F,382,"[849928, 847375, 849938, 849946, 849436, 84943...",1,2023-04-20 00:11:00.962160,[],[],N,0,0,0,37,356.0
1252,839357,0,0,M,2299,"[855552, 857473, 861830, 866953, 867849, 86605...",1,2023-04-20 14:26:20.479856,[],[],N,0,1,0,52,519.0
1497,839670,0,0,F,643,"[871299, 850180, 850948, 849670, 851593, 84737...",0,2023-04-21 00:25:27.279382,[],[],N,0,1,0,24,603.0


In [59]:
# 연-월-일 컬럼 뽑기
user_target_df['year_month_day'] = user_target_df['created_at'].dt.strftime('%Y-%m-%d')
user_target_df.head()

/tmp/ipykernel_679393/1907878019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_target_df['year_month_day'] = user_target_df['created_at'].dt.strftime('%Y-%m-%d')


,id,is_superuser,is_staff,gender,point,friend_id_list,is_push_on,created_at,block_user_id_list,hide_user_id_list,ban_status,report_count,alarm_count,pending_chat,pending_votes,group_id,year_month_day
396,838023,0,0,F,2456,"[855552, 861830, 859783, 850186, 868492, 85595...",1,2023-04-19 09:06:00.719792,[],[],N,0,1,0,115,233.0,2023-04-19
633,838466,0,0,F,1680,"[876288, 869250, 873349, 867975, 878602, 11573...",1,2023-04-19 15:10:56.560337,[],[],N,0,1,0,244,321.0,2023-04-19
756,838642,0,0,F,382,"[849928, 847375, 849938, 849946, 849436, 84943...",1,2023-04-20 00:11:00.962160,[],[],N,0,0,0,37,356.0,2023-04-20
1252,839357,0,0,M,2299,"[855552, 857473, 861830, 866953, 867849, 86605...",1,2023-04-20 14:26:20.479856,[],[],N,0,1,0,52,519.0,2023-04-20
1497,839670,0,0,F,643,"[871299, 850180, 850948, 849670, 851593, 84737...",0,2023-04-21 00:25:27.279382,[],[],N,0,1,0,24,603.0,2023-04-21


In [60]:
# 일별 회원가입 유저 수 구하기
user_count_df = user_target_df.groupby(['year_month_day'])['id'].nunique().reset_index(name='daily_user_cnt')
user_count_df.head()

,year_month_day,daily_user_cnt
0,2023-04-19,2
1,2023-04-20,2
2,2023-04-21,6
3,2023-04-22,4
4,2023-04-24,5


In [61]:
# 누적 회원 수 구하기
user_count_df['total_user'] = user_count_df['daily_user_cnt'].cumsum()
user_count_df.head()

,year_month_day,daily_user_cnt,total_user
0,2023-04-19,2,2
1,2023-04-20,2,4
2,2023-04-21,6,10
3,2023-04-22,4,14
4,2023-04-24,5,19


In [62]:
# 5월부터 확인해보기
user_count_df = user_count_df[user_count_df['year_month_day'] >= '2023-05-01']
user_count_df.head()

,year_month_day,daily_user_cnt,total_user
11,2023-05-01,43,446
12,2023-05-02,99,545
13,2023-05-03,389,934
14,2023-05-04,184,1118
15,2023-05-05,434,1552


---

### 일별 투표자 수 확인하기

In [68]:
userquestion_school_567_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1181418 entries, 30798 to 1212215
Data columns (total 15 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   status             1181418 non-null  object        
 1   created_at         1181418 non-null  datetime64[ns]
 2   chosen_user_id     1181418 non-null  int64         
 3   question_id        1181418 non-null  int64         
 4   user_id            1181418 non-null  int64         
 5   question_piece_id  1181418 non-null  int64         
 6   has_read           1181418 non-null  int64         
 7   answer_status      1181418 non-null  object        
 8   answer_updated_at  1181418 non-null  datetime64[ns]
 9   group_id           1181418 non-null  float64       
 10  grade              1181418 non-null  float64       
 11  school_id          1181418 non-null  float64       
 12  address            1181418 non-null  object        
 13  student_count      1181418 n

In [70]:
# 연-월-일 컬럼 만들기
userquestion_school_567_df['year_month_day'] = userquestion_school_567_df['created_at'].dt.strftime('%Y-%m-%d')

# 일별 투표자 수 구하기
daily_vote_user_cnt_df = userquestion_school_567_df.groupby(['year_month_day'])['user_id'].nunique().reset_index(name='daily_vote_user_cnt')
daily_vote_user_cnt_df.head()

,year_month_day,daily_vote_user_cnt
0,2023-05-01,325
1,2023-05-02,413
2,2023-05-03,756
3,2023-05-04,820
4,2023-05-05,1248


In [71]:
# user_count_df 와 합치기
user_count_df = pd.merge(user_count_df, daily_vote_user_cnt_df, on='year_month_day',how='left')
user_count_df.head()

,year_month_day,daily_user_cnt,total_user,daily_vote_user_cnt
0,2023-05-01,43,446,325.0
1,2023-05-02,99,545,413.0
2,2023-05-03,389,934,756.0
3,2023-05-04,184,1118,820.0
4,2023-05-05,434,1552,1248.0


In [72]:
# 투표율 구하기
user_count_df['vote_rate'] = user_count_df['daily_vote_user_cnt'] / user_count_df['total_user']
user_count_df.head()

,year_month_day,daily_user_cnt,total_user,daily_vote_user_cnt,vote_rate
0,2023-05-01,43,446,325.0,0.728700
1,2023-05-02,99,545,413.0,0.757798
2,2023-05-03,389,934,756.0,0.809422
3,2023-05-04,184,1118,820.0,0.733453
4,2023-05-05,434,1552,1248.0,0.804124


---